## Imports

In [32]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os
import geopandas as gpd

## Read-in

In [33]:
df = pd.read_csv('lidia_dev_map - Sheet1.csv')

## Image URL collector

In [17]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [18]:
# df['image_links'] = ''
# links = []
# for url in df['story_link']:
#     try:
#         page = requests.get(url)
#         soup = BeautifulSoup(page.content, 'html.parser')
#         images = soup.find_all('img')
#         counter = 0
#         for image in images:
#             counter += 1
#             if 'jpg' in image['src']:
#                 links.append(image['src'])
#                 if counter == len(images):
#                     links.append('no image found')
#                 break
#     except:
#         links.append('no_image')

# ### Had to append an image because of png ###
# links.append('https://s14.therealdeal.com/trd/m/up/2021/10/main-MIA-Location-Ventures-co-living_10.13.21-705x439.png')
# df['image_links'] = links

## Article link formatter

In [19]:
# df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
# df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
# df['remaining_desc']

# df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
# df['description_link']

## Google Maps API Geolocater Setup

In [20]:
# %store -r google_maps_API_Key
# geolocator = GoogleV3(api_key=google_maps_API_Key)

In [21]:
# df['geo_address'] = df['full address'] + ' Miami Dade, FL'
# df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
# df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
# df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [34]:
df.columns

Index(['developers', 'description', 'lat', 'lon'], dtype='object')

In [35]:
df

,developers,description,lat,lon
0,BH Group and Avy Shlomo,"9-story, 132-unit rental building",25.946427,-80.152198
1,Prologis,"4 warehouses totaling over 500,000 square feet",25.779337,-80.421250
2,McDowell Housing Partners,10-story building with 208 affordable apartments,25.568536,-80.381124


## Correction section

In [24]:
# df.at[4,'lat']=('25.7793373934833')
# df.at[4,'lon']=('-80.4212499653117')

# df.at[5,'lat']=('25.5685360321924')
# df.at[5,'lon']=('-80.381124219578')

## HTML popup formatter

In [25]:
df.columns

Index(['full address', 'developers', 'description', 'lat', 'lon',
       'geo_address', 'loc', 'point', 'altitude'],
      dtype='object')

In [36]:
def popup_html(row):
    i = row
    developers = df['developers'].iloc[i]
    description = df['description'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Developers: <strong>{}'''.format(developers) + '''</strong><br>
    <strong>{}'''.format(description) + '''</strong><br>
    </html>
    '''
    return html


### HTML reservoir

In [27]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [37]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=13)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"Miami-Dade Dev Flurry")

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [ ]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [ ]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [ ]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)